In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load datasets
def load_data(file_path, dataset_type):
    df = pd.read_csv(file_path, sep=";", header=None)
    df.columns = ["Text", "Emotions"]
    df['Dataset'] = dataset_type
    return df

In [3]:
train_df = load_data("./dataset/train.txt", 'Train')
test_df = load_data("./dataset/test.txt", 'Test')
val_df = load_data("./dataset/val.txt", 'Validation')

In [4]:
# Combine datasets
text_emotions_combined_df = pd.concat([train_df, test_df, val_df], ignore_index=True)

In [5]:
# Save the combined data to a new text file
text_emotions_combined_df.to_csv("./dataset/text_emotions_combined_dataset.txt", sep=";", index=False)

print("Text Emotions Combined data has been saved to 'text_emotions_combined_dataset.txt'.")

Text Emotions Combined data has been saved to 'text_emotions_combined_dataset.txt'.


In [6]:
# Data preprocessing
texts = text_emotions_combined_df["Text"].tolist()
labels = text_emotions_combined_df["Emotions"].tolist()

In [7]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [8]:
# Encode and one-hot encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
one_hot_labels = keras.utils.to_categorical(labels)

In [9]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state = 42)

In [10]:
# Model architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

In [11]:
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [12]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)

In [13]:
# Train model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test),
          callbacks=[early_stopping, model_checkpoint])

Epoch 1/5
500/500 [==============================] - 341s 635ms/step - loss: 0.8907 - accuracy: 0.6678 - val_loss: 0.3159 - val_accuracy: 0.8882
Epoch 2/5
500/500 [==============================] - 200s 398ms/step - loss: 0.1794 - accuracy: 0.9361 - val_loss: 0.2533 - val_accuracy: 0.9095
Epoch 3/5
500/500 [==============================] - 281s 562ms/step - loss: 0.0850 - accuracy: 0.9678 - val_loss: 0.2776 - val_accuracy: 0.9107
Epoch 4/5
500/500 [==============================] - 247s 494ms/step - loss: 0.0564 - accuracy: 0.9796 - val_loss: 0.3217 - val_accuracy: 0.9082
Epoch 5/5
500/500 [==============================] - 164s 327ms/step - loss: 0.0499 - accuracy: 0.9820 - val_loss: 0.3322 - val_accuracy: 0.9055


In [14]:
# Prediction
input_text = "I am very happy because code is running!"
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])

1/1 [==============================] - 18s 18s/step


In [15]:
print("Predicted Emotion:", predicted_label[0])

Predicted Emotion: joy


In [16]:
# Save the tokenizer and label encoder for later use
#with open("./model/tokenizer.pkl", "wb") as tokenizer_file:
   # pickle.dump(tokenizer, tokenizer_file)

#with open("./model/label_encoder.pkl", "wb") as label_encoder_file:
   # pickle.dump(label_encoder, label_encoder_file)

In [17]:
# Save model parameters
#model_params = {
 #   "max_length": max_length,
 #   "num_classes": len(one_hot_labels[0])
#}

In [18]:
#with open("./model/model_params.pkl", "wb") as model_params_file:
 #   pickle.dump(model_params, model_params_file)

In [19]:
# print("Model, tokenizer, and label encoder saved successfully!")